# Strands Agents Evaluation Framework

## Overview

This notebook demonstrates how to use the Strands Agents Evaluation SDK to evaluate agent performance. We'll use the same web search agent from Module 3 (Agentic Metrics) and evaluate it using Strands Evals' built-in evaluators.

### What You'll Learn

- How to install and configure the Strands Evals SDK
- Creating test cases with the `Case` class
- Using `OutputEvaluator` to assess response quality
- Using `TrajectoryEvaluator` to evaluate tool usage
- Running experiments and analyzing results
- Building custom evaluators for domain-specific needs

### Why Strands Evals?

Strands Evals provides a structured approach to agent evaluation that goes beyond simple accuracy metrics:

- **LLM-as-Judge**: Uses powerful models to evaluate response quality against custom rubrics
- **Trajectory Evaluation**: Assesses whether agents use the right tools in the right order
- **Experiment Management**: Organize test cases, run evaluations, and track results over time
- **Interactive Reports**: Visual display of evaluation results with detailed reasoning

## 1. Dependencies Installation

In [3]:
!pip install strands-agents strands-agents-tools strands-agents-evals --quiet
!pip install ddgs --quiet
!pip install pandas --quiet
!pip install typing-extensions --upgrade --quiet

## 2. Imports

In [4]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands_evals import Case, Experiment
from strands_evals.evaluators import OutputEvaluator, TrajectoryEvaluator
from strands_evals.extractors import tools_use_extractor
from bs4 import BeautifulSoup
import requests
import pandas as pd
from botocore.config import Config
from ddgs import DDGS

## 3. Ground Truth Dataset Loading

We'll use the same city population dataset from Module 3. This provides our gold standard for evaluating agent accuracy.

In [5]:
# Read the CSV file containing city, state, population, and land area data
gold_standard_city_pop = pd.read_csv('city_pop.csv')

# Clean city names (remove Wikipedia footnote brackets)
gold_standard_city_pop['city'] = gold_standard_city_pop['city'].str.replace(r'\[.*?\]', '', regex=True)

# Clean numeric columns (remove commas)
gold_standard_city_pop['population'] = gold_standard_city_pop['population'].astype(str).str.replace(',', '').astype(float)
gold_standard_city_pop['land_area_mi2'] = gold_standard_city_pop['land_area_mi2'].astype(str).str.replace(',', '').astype(float)

print(gold_standard_city_pop.head(3))

          city state  population  land_area_mi2
0     New York    NY   8478072.0          300.5
1  Los Angeles    CA   3878704.0          469.5
2      Chicago    IL   2721308.0          227.7


## 4. Agent Tools

These tools enable web-based information retrieval for the agent, identical to Module 3.

- `web_search`: Queries DuckDuckGo to find relevant web pages
- `get_page`: Extracts raw text content from specific URLs

In [6]:
@tool
def web_search(topic: str) -> str:
    """Search DuckDuckGo for a given topic."""
    try:
        results = DDGS().text(topic, max_results=5)
        
        if not results:
            return "No search results found"
        
        result_string = ""
        for i, result in enumerate(results):
            result_string += f"Result {i+1}: {result.get('title', 'No title')}\nURL: {result.get('href', 'No URL')}\nSnippet: {result.get('body', 'No description')}\n\n"
        
        return result_string
        
    except Exception as e:
        return f"Search error: {str(e)}"

@tool
def get_page(url: str) -> str:
    """Fetch and return the raw text from a URL."""
    response = requests.get(url)
    response.raise_for_status()
    bs = BeautifulSoup(response.text, 'html.parser')
    return bs.text

## 5. AWS Bedrock Configuration

Configure Bedrock API calls with appropriate timeouts for evaluation scenarios.

In [7]:
# Quick config for rapid evaluation runs
quick_config = Config(
    connect_timeout=5,
    read_timeout=30,
    retries={"max_attempts": 1}
)

# Model configuration
model_name = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
chatbot_model = BedrockModel(
    model_id=model_name,
    boto_client_config=quick_config
)

## 6. Create the Web Search Agent

We create a tour guide agent that answers questions about city populations and areas. The agent uses structured XML tags in its output for programmatic parsing.

In [8]:
system_prompt = '''You are a helpful tour guide. Customers may ask you about the population and size of cities.
You should use tools to retrieve all data, to make sure it is as current as possible.
Please put your human friendly response in 'response' XML tags, and then follow with your data results in 'pop' and 'area' XML tags, for programatic processing.
The values in the 'pop' and 'area' XML tags should only be numbers, no words or commas.
You may also include a fun fact about the city in question.
For example, your response to "what is the population and size of Somewhereland?" might be:
<response>Somewhereland is a great place to get a hot dog. It has a population of 3000 people, and is 100 square miles.</response>
<pop>3000</pop><area>100</area>
'''

chatbot = Agent(
    system_prompt=system_prompt,
    tools=[web_search, get_page],
    model=chatbot_model
)

### Test the Agent

Let's verify the agent works correctly before setting up evaluations.

In [23]:
import json

# Custom callback handler to display tool calls with their parameters.
# 
# Strands agents stream output via callbacks. The callback receives kwargs including:
# - 'data': streaming text chunks from the model's response
# - 'current_tool_use': info about a tool being called (name, toolUseId, input)
#
# The challenge: tool input arrives as a streaming JSON string, so the callback fires
# multiple times as each character arrives. We detect completion by attempting to parse
# the JSON - if it parses successfully, the input is complete.
#
# We track toolUseId (unique per invocation) in a set to avoid duplicate prints.
# This allows the same tool to be called multiple times - each gets a unique ID.

seen_complete_tools = set()

def verbose_callback(**kwargs):
    """Callback handler that displays streaming text and tool calls with parameters."""
    # Stream text output as it arrives
    if 'data' in kwargs:
        print(kwargs['data'], end='', flush=True)
    
    # Display tool calls when their input is complete
    if 'current_tool_use' in kwargs:
        tool_use = kwargs['current_tool_use']
        tool_id = tool_use.get('toolUseId', 'no-id')
        tool_name = tool_use.get('name', 'no-name')
        tool_input = tool_use.get('input', '')
        
        # Skip if we've already printed this tool call
        if tool_id in seen_complete_tools:
            return
            
        # Try to parse the input as JSON - if successful, it's complete
        if isinstance(tool_input, str) and tool_input:
            try:
                parsed_input = json.loads(tool_input)
                seen_complete_tools.add(tool_id)
                print(f"\n\nTool Call: {tool_name} {json.dumps(parsed_input, indent=2)}")
                print()
            except json.JSONDecodeError:
                pass  # Input is still streaming, not yet valid JSON

In [24]:

# Create agent with debug handler
debug_chatbot = Agent(
    system_prompt=system_prompt,
    tools=[web_search, get_page],
    model=chatbot_model,
    callback_handler=debug_callback
)

prompt = "How many people live in New York, and what's the area of the city in square miles?"
chatbot_response = debug_chatbot(prompt)



Tool Call: web_search {
  "topic": "New York City population and area square miles"
}

<response>
New York City is the most densely populated major city in the United States! It has a population of approximately 8,478,072 people and covers an area of 300.46 square miles.
</response>
<pop>8478072</pop>
<area>300.46</area>

## 7. Introduction to Strands Evals

Strands Evals uses three core concepts:

1. **Case**: A test case containing input, expected output, and metadata
2. **Evaluator**: Assesses agent responses against criteria (rubrics)
3. **Experiment**: A collection of test cases with evaluators that runs evaluations

Let's start with a simple output evaluation.

## 8. Basic Output Evaluation

The `OutputEvaluator` uses an LLM judge to assess response quality against a custom rubric. This is useful for evaluating whether the agent's output meets specific criteria.

### Create Test Cases

We'll create test cases from our gold standard dataset. Each case includes:
- `name`: Unique identifier for the test
- `input`: The prompt to send to the agent
- `expected_output`: What we expect the agent to return
- `metadata`: Additional context for categorization

In [25]:
# Create test cases from the first 3 cities in our dataset
test_cases = []
for idx, row in gold_standard_city_pop.head(3).iterrows():
    test_cases.append(
        Case[str, str](
            name=f"city-{idx+1}-{row['city'].lower().replace(' ', '-')}",
            input=f"What is the population and area in square miles of {row['city']}, {row['state']}?",
            expected_output=f"Population: {int(row['population'])}, Area: {row['land_area_mi2']} square miles",
            metadata={"category": "city_info", "city": row['city'], "state": row['state']}
        )
    )

# Display the test cases
for case in test_cases:
    print(f"Test: {case.name}")
    print(f"  Input: {case.input}")
    print(f"  Expected: {case.expected_output}\n")

Test: city-1-new-york
  Input: What is the population and area in square miles of New York, NY?
  Expected: Population: 8478072, Area: 300.5 square miles

Test: city-2-los-angeles
  Input: What is the population and area in square miles of Los Angeles, CA?
  Expected: Population: 3878704, Area: 469.5 square miles

Test: city-3-chicago
  Input: What is the population and area in square miles of Chicago, IL?
  Expected: Population: 2721308, Area: 227.7 square miles



### Create an Output Evaluator

The evaluator uses a rubric to guide the LLM judge. The rubric defines what constitutes a good response.

In [26]:
# Create an evaluator with a custom rubric
accuracy_evaluator = OutputEvaluator(
    rubric="""
    Evaluate the response based on:
    1. Population Accuracy - Is the population value close to the expected value? (within 5% is excellent, within 10% is acceptable)
    2. Area Accuracy - Is the area value close to the expected value? (within 5% is excellent, within 10% is acceptable)
    3. Completeness - Does the response include both population AND area information?
    
    Score 1.0 if both values are within 5% of expected and response is complete.
    Score 0.7 if both values are within 10% of expected.
    Score 0.5 if only one value is accurate or response is incomplete.
    Score 0.0 if values are significantly wrong or missing.
    
    Focus on the numerical accuracy, not the exact format of the response.
    """,
    include_inputs=True
)

print("Output evaluator created with accuracy rubric")

Output evaluator created with accuracy rubric


### Define the Task Function

The task function takes a test case and returns the agent's response. This is what gets evaluated.

In [27]:
def get_agent_response(case: Case) -> str:
    """Run the agent on a test case and return the response."""
    # Create a fresh agent for each test to avoid context contamination
    agent = Agent(
        system_prompt=system_prompt,
        tools=[web_search, get_page],
        model=chatbot_model,
        callback_handler=None  # Suppress output during evaluation
    )
    response = agent(case.input)
    return str(response)

### Run the Experiment

Create an experiment with our test cases and evaluator, then run the evaluations.

In [29]:
# Create the experiment
experiment = Experiment[str, str](
    cases=test_cases,
    evaluators=[accuracy_evaluator]
)

# Run evaluations
print("Running output evaluation experiment...")
print("This may take a few minutes as the agent searches the web for each city.\n")

reports = experiment.run_evaluations(get_agent_response)

# Display results
print("\n=== Output Evaluation Results ===")
reports[0].run_display()

Running output evaluation experiment...
This may take a few minutes as the agent searches the web for each city.


=== Output Evaluation Results ===


╭───────────────────────────────────────────── 📊 Evaluation Report ──────────────────────────────────────────────╮
│ Overall Score: 0.90           Pass Rate: 1.0                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                         Test Case Results                         
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ index ┃ name               ┃ score ┃ test_pass ┃ reason ┃ input ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ ▶ 0   │ city-1-new-york    │ 1.00  │ ✅        │ ...    │ ...   │
├───────┼────────────────────┼───────┼───────────┼────────┼───────┤
│ ▶ 1   │ city-2-los-angeles │ 0.70  │ ✅        │ ...    │ ...   │
├───────┼────────────────────┼───────┼───────────┼────────┼───────┤
│ ▶ 2   │ city-3-chicago     │ 1.00  │ ✅        │ ...    │ ...   │
└───────┴────────────────────┴───────┴───────────┴────────┴───────┘

Enter the test case number to expand/collapse it, o to expand all, and c to collapse all (q to quit).:

## 9. Trajectory Evaluation (Tool Usage)

Beyond output quality, we often want to evaluate HOW the agent solved the problem. The `TrajectoryEvaluator` assesses whether the agent used the right tools in the right order.

This is particularly important for:
- Ensuring agents follow expected workflows
- Detecting inefficient tool usage patterns
- Validating that agents use appropriate tools for specific tasks

### Create Test Cases with Expected Trajectories

For trajectory evaluation, we specify which tools we expect the agent to use.

In [30]:
# Test cases with expected tool usage
trajectory_test_cases = [
    Case[str, str](
        name="web-search-required",
        input="What is the population of Phoenix, AZ?",
        expected_trajectory=["web_search"],  # We expect the agent to search the web
        metadata={"category": "tool_usage", "expected_tools": ["web_search"]}
    ),
    Case[str, str](
        name="multi-tool-query",
        input="Find detailed information about Houston, TX population from an official source.",
        expected_trajectory=["web_search", "get_page"],  # Search then fetch page
        metadata={"category": "tool_usage", "expected_tools": ["web_search", "get_page"]}
    )
]

print("Trajectory test cases created:")
for case in trajectory_test_cases:
    print(f"  {case.name}: expects {case.expected_trajectory}")

Trajectory test cases created:
  web-search-required: expects ['web_search']
  multi-tool-query: expects ['web_search', 'get_page']


### Create a Trajectory Evaluator

In [31]:
# Create trajectory evaluator
trajectory_evaluator = TrajectoryEvaluator(
    rubric="""
    Evaluate the tool usage trajectory:
    1. Correct tool selection - Were the expected tools used?
    2. Logical sequence - Were tools used in a sensible order?
    3. Efficiency - Were unnecessary tools avoided?
    
    Score 1.0 if all expected tools were used appropriately.
    Score 0.7 if expected tools were used but with some inefficiency.
    Score 0.5 if some expected tools were missing but task was completed.
    Score 0.0 if wrong tools were used or critical tools were missing.
    """,
    include_inputs=True
)

# Update evaluator with tool descriptions for context
sample_agent = Agent(tools=[web_search, get_page], callback_handler=None)
tool_descriptions = tools_use_extractor.extract_tools_description(sample_agent, is_short=True)
trajectory_evaluator.update_trajectory_description(tool_descriptions)

print("Trajectory evaluator created")

Trajectory evaluator created


### Define Task Function with Trajectory Capture

In [32]:
def get_response_with_trajectory(case: Case) -> dict:
    """Run the agent and capture both output and tool usage trajectory."""
    agent = Agent(
        system_prompt=system_prompt,
        tools=[web_search, get_page],
        model=chatbot_model,
        callback_handler=None
    )
    response = agent(case.input)
    
    # Extract the tools that were used
    trajectory = tools_use_extractor.extract_agent_tools_used_from_messages(agent.messages)
    
    return {
        "output": str(response),
        "trajectory": trajectory
    }

### Run Trajectory Evaluation

In [33]:
# Create and run trajectory experiment
trajectory_experiment = Experiment[str, str](
    cases=trajectory_test_cases,
    evaluators=[trajectory_evaluator]
)

print("Running trajectory evaluation experiment...\n")
trajectory_reports = trajectory_experiment.run_evaluations(get_response_with_trajectory)

print("\n=== Trajectory Evaluation Results ===")
trajectory_reports[0].run_display()

Running trajectory evaluation experiment...


=== Trajectory Evaluation Results ===


╭───────────────────────────────────────────── 📊 Evaluation Report ──────────────────────────────────────────────╮
│ Overall Score: 0.85           Pass Rate: 1.0                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                         Test Case Results                          
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ index ┃ name                ┃ score ┃ test_pass ┃ reason ┃ input ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ ▶ 0   │ web-search-required │ 1.00  │ ✅        │ ...    │ ...   │
├───────┼─────────────────────┼───────┼───────────┼────────┼───────┤
│ ▶ 1   │ multi-tool-query    │ 0.70  │ ✅        │ ...    │ ...   │
└───────┴─────────────────────┴───────┴───────────┴────────┴───────┘

Enter the test case number to expand/collapse it, o to expand all, and c to collapse all (q to quit).:

## 10. Custom Evaluators

For domain-specific evaluation needs, you can create custom evaluators. This example creates an evaluator that checks if the agent's response contains properly formatted XML tags.

In [34]:
import re
from strands_evals.evaluators import Evaluator
from strands_evals.types import EvaluationData, EvaluationOutput

class XMLFormatEvaluator(Evaluator[str, str]):
    """Custom evaluator that checks for proper XML tag formatting in responses."""
    
    def evaluate(self, evaluation_case: EvaluationData[str, str]) -> EvaluationOutput:
        response = evaluation_case.actual_output
        
        # Check for required XML tags
        has_response_tag = bool(re.search(r'<response>.*?</response>', response, re.DOTALL))
        has_pop_tag = bool(re.search(r'<pop>\d+</pop>', response))
        has_area_tag = bool(re.search(r'<area>[\d.]+</area>', response))
        
        # Calculate score based on tag presence
        tags_present = sum([has_response_tag, has_pop_tag, has_area_tag])
        score = tags_present / 3.0
        
        # Build reason
        missing = []
        if not has_response_tag:
            missing.append("<response>")
        if not has_pop_tag:
            missing.append("<pop>")
        if not has_area_tag:
            missing.append("<area>")
        
        if score == 1.0:
            reason = "All required XML tags present and properly formatted"
            label = "complete"
        else:
            reason = f"Missing or malformed tags: {', '.join(missing)}"
            label = "incomplete"
        
        return EvaluationOutput(
            score=score,
            test_pass=score >= 0.66,  # Pass if at least 2/3 tags present
            reason=reason,
            label=label
        )

print("Custom XMLFormatEvaluator created")

Custom XMLFormatEvaluator created


### Run Evaluation with Custom Evaluator

In [35]:
# Create test cases for format evaluation
format_test_cases = [
    Case[str, str](
        name="format-test-1",
        input="What is the population and area of Chicago, IL?",
        metadata={"category": "format_check"}
    ),
    Case[str, str](
        name="format-test-2",
        input="Tell me about Los Angeles, CA - population and area?",
        metadata={"category": "format_check"}
    )
]

# Create experiment with custom evaluator
format_evaluator = XMLFormatEvaluator()
format_experiment = Experiment[str, str](
    cases=format_test_cases,
    evaluators=[format_evaluator]
)

print("Running format evaluation experiment...\n")
format_reports = format_experiment.run_evaluations(get_agent_response)

print("\n=== Format Evaluation Results ===")
format_reports[0].run_display()

Running format evaluation experiment...


=== Format Evaluation Results ===


╭───────────────────────────────────────────── 📊 Evaluation Report ──────────────────────────────────────────────╮
│ Overall Score: 0.00           Pass Rate: 0.0                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                      Test Case Results                       
┏━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ index ┃ name          ┃ score ┃ test_pass ┃ reason ┃ input ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ ▶ 0   │ format-test-1 │ 0.00  │ ❌        │ ...    │ ...   │
├───────┼───────────────┼───────┼───────────┼────────┼───────┤
│ ▶ 1   │ format-test-2 │ 0.00  │ ❌        │ ...    │ ...   │
└───────┴───────────────┴───────┴───────────┴────────┴───────┘

Enter the test case number to expand/collapse it, o to expand all, and c to collapse all (q to quit).:

## 11. Combining Multiple Evaluators

For comprehensive evaluation, you can combine multiple evaluators in a single experiment. This provides a multi-dimensional view of agent performance.

In [36]:
# Create a comprehensive test case
comprehensive_test_cases = [
    Case[str, str](
        name="comprehensive-seattle",
        input="What is the population and area in square miles of Seattle, WA?",
        expected_output="Population: 780995, Area: 83.8 square miles",
        expected_trajectory=["web_search"],
        metadata={"category": "comprehensive", "city": "Seattle"}
    )
]

# Combine evaluators
comprehensive_experiment = Experiment[str, str](
    cases=comprehensive_test_cases,
    evaluators=[
        accuracy_evaluator,      # Check output accuracy
        trajectory_evaluator,    # Check tool usage
        format_evaluator         # Check XML formatting
    ]
)

print("Running comprehensive evaluation with multiple evaluators...\n")
comprehensive_reports = comprehensive_experiment.run_evaluations(get_response_with_trajectory)

print("\n=== Comprehensive Evaluation Results ===")
for i, report in enumerate(comprehensive_reports):
    print(f"\nEvaluator {i+1} Results:")
    report.run_display()

Running comprehensive evaluation with multiple evaluators...


=== Comprehensive Evaluation Results ===

Evaluator 1 Results:


╭───────────────────────────────────────────── 📊 Evaluation Report ──────────────────────────────────────────────╮
│ Overall Score: 0.70           Pass Rate: 1.0                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                          Test Case Results                           
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ index ┃ name                  ┃ score ┃ test_pass ┃ reason ┃ input ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ ▶ 0   │ comprehensive-seattle │ 0.70  │ ✅        │ ...    │ ...   │
└───────┴───────────────────────┴───────┴───────────┴────────┴───────┘

Enter the test case number to expand/collapse it, o to expand all, and c to collapse all (q to quit).:


Evaluator 2 Results:


...

Enter the test case number to expand/collapse it, o to expand all, and c to collapse all (q to quit).:


Evaluator 3 Results:


╭───────────────────────────────────────────── 📊 Evaluation Report ──────────────────────────────────────────────╮
│ Overall Score: 0.00           Pass Rate: 0.0                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                          Test Case Results                           
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ index ┃ name                  ┃ score ┃ test_pass ┃ reason ┃ input ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ ▶ 0   │ comprehensive-seattle │ 0.00  │ ❌        │ ...    │ ...   │
└───────┴───────────────────────┴───────┴───────────┴────────┴───────┘

Enter the test case number to expand/collapse it, o to expand all, and c to collapse all (q to quit).:

## 12. Saving and Loading Experiments

Strands Evals supports saving experiments for later analysis or comparison.

In [38]:
# Save the experiment to a file
experiment.to_file("city_population_evaluation.json")
print("Experiment saved to city_population_evaluation.json")

# You can load it later with:
# loaded_experiment = Experiment.from_file("city_population_evaluation.json")

Experiment saved to city_population_evaluation.json


## Summary

In this notebook, we've covered the core capabilities of the Strands Agents Evaluation SDK:

1. **OutputEvaluator**: LLM-as-judge evaluation of response quality using custom rubrics
2. **TrajectoryEvaluator**: Assessment of tool usage patterns and sequences
3. **Custom Evaluators**: Domain-specific evaluation logic for specialized needs
4. **Experiment Management**: Organizing test cases and running evaluations

### Key Takeaways

- Use `Case` objects to define test inputs, expected outputs, and expected tool trajectories
- Rubrics guide the LLM judge - be specific about scoring criteria
- Combine multiple evaluators for comprehensive assessment
- Save experiments for tracking performance over time

### Next Steps

- Explore `HelpfulnessEvaluator` for assessing overall agent helpfulness
- Try `GoalSuccessRateEvaluator` for end-to-end goal achievement
- Use async evaluation with `run_evaluations_async()` for better performance
- Check out the [Strands Evals documentation](https://strandsagents.com/latest/documentation/docs/user-guide/evals-sdk/) for more advanced features